In [1]:
import numpy as np 
import open3d as o3d 
import mrob 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from sklearn.cluster import AgglomerativeClustering

In [3]:
import sys
import pathlib
import copy

sys.path.append("..")

In [4]:
import voxel_slam

In [5]:
def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            if line.startswith('#'):
                continue
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=-1, center_distance_threshold=1.5):
    poses = []
    clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidar_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidar_t).T()

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > center_distance_threshold)[0] 

        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])

        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        
        poses.append(pose)
        clouds.append(cloud)

    return clouds, poses

In [6]:
clouds_path = "/home/ach/Desktop/datasets/hilti/out2"
poses_path = "/home/ach/Desktop/datasets/hilti/exp14_basement_2_imu.txt"
imu_path = "/home/ach/Desktop/datasets/hilti/hilti_imu.txt"

In [7]:
ts_multiplier = 1 / 1e9
ts_to_depth_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)
ts_to_imu = parse_trajectories_float(imu_path, ts_multiplier=ts_multiplier)

In [8]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=320, number_of_clouds=3, center_distance_threshold=3)

In [9]:
voxel_map = voxel_slam.VoxelMap(clouds, poses, voxel_size=2.0)

In [10]:
feature_map = voxel_map.adaptive_feature_extraction(ransac_distance_threshold=0.02, adaptive_voxel_size=0.5)

## Test optimization

In [11]:
def break_on_minimaps(clouds, poses, minimap_size=5, adaptive_voxelisation_threshold=0.5):
    transformed_clouds = [None for _ in range(len(poses))]
    for i in range(len(poses)):
        transformed_clouds[i] = copy.deepcopy(clouds[i]).transform(poses[i])

    optimized_submaps = []
    for i in range(0, len(poses), minimap_size):
        voxel_map = voxel_slam.VoxelMap(
            transformed_clouds[i:i+minimap_size],
            [np.eye(4) for _ in range(minimap_size)],
            voxel_size=2.0
        )
        feature_map = voxel_map.adaptive_feature_extraction(
            ransac_distance_threshold=0.02,
            adaptive_voxel_size=adaptive_voxelisation_threshold
        )

        print(f"Submap {i}-{i+minimap_size}:", end='\n')
        opt_poses, is_converged, chi2 = voxel_slam.BaregBackend(feature_map, minimap_size).get_optimized_poses(1000, verbose=True)
        print("====")

        optimized_submaps.append(
            voxel_slam.aggregate_map(voxel_map.transformed_clouds, opt_poses, enable_color=True)
        )

    aggregate_filter = voxel_slam.EmptyVoxelsFilter(min_voxel_poses=2)
    aggregate_filter.set_next(voxel_slam.NormalsFilter(0.2))

    print("Aggregated map:", end=' ')
    aggregate_pipeline = voxel_slam.VoxelSLAMPipeline(
        feature_filter=aggregate_filter,
        optimization_backend=voxel_slam.BaregBackend,
        config=voxel_slam.PipelineConfig(voxel_size=2.0, 
                                         ransac_distance_threshold=0.02, 
                                         filter_cosine_distance_threshold=0.2,
                                         backend_verbose=True)
    )
    o3d.visualization.draw_geometries(optimized_submaps)
    aggregate_output = aggregate_pipeline.process(optimized_submaps, [np.eye(4) for _ in range(len(optimized_submaps))])
    
    o3d.visualization.draw_geometries([
        voxel_slam.aggregate_map(aggregate_output.optimized_clouds, aggregate_output.optimized_poses)
    ])

In [12]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=320, number_of_clouds=30, center_distance_threshold=3)

In [13]:
o3d.visualization.draw_geometries([
    voxel_slam.aggregate_map(clouds, poses, enable_color=True)
])

In [14]:
break_on_minimaps(clouds, poses, minimap_size=5, adaptive_voxelisation_threshold=0.5)

Submap 0-5:
FGraph initial error: 8863.640852269682
Iteratios to converge: 92
Chi2: 1189.1241030931985
====
Submap 5-10:
FGraph initial error: 51178.067838570714
Iteratios to converge: 89
Chi2: 46074.096293640345
====
Submap 10-15:
FGraph initial error: 13673.870279291086
Iteratios to converge: 72
Chi2: 1785.4909717129017
====
Submap 15-20:
FGraph initial error: 14810.942868621756
Iteratios to converge: 60
Chi2: 2216.5559848289013
====
Submap 20-25:
FGraph initial error: 14909.686958787852
Iteratios to converge: 0
Chi2: 3603.3454928818146
====
Submap 25-30:
FGraph initial error: 6065.093159287223
Iteratios to converge: 51
Chi2: 1260.3250007829279
====
Aggregated map: FGraph initial error: 1146143.3788600704
Iteratios to converge: 56
Chi2: 1096080.9768293248
